## 5.3 Starvation
A process or thread is perpetually denied the resource it needs. In a simple scenario with two equal threads competing for execution time, starvation probably isn't a concern. Both threads should get plenty of chances to acquire the resource. However, if two threads are given different priorities, then that may not be the case. How different thread priorities get treated will depend on the operating system. But generally higher priority threads will be scheduled to execute more often, and that can leave low priority threads feeling hungry. Another thing that can lead to starvation is having too many concurrent threads.

To demonstrate thread starvation in Python, we'll modify the dining philosophers example by adding a local variable within the `philosopher` function to keep track of how many pieces of sushi each philosopher thread gets to eat.

In [4]:
#!/usr/bin/env python3
""" Three philosophers, thinking and eating sushi """

import threading

chopstick_a = threading.Lock()
chopstick_b = threading.Lock()
chopstick_c = threading.Lock()
sushi_count = 5000

def philosopher(name, first_chopstick, second_chopstick):
    global sushi_count
    sushi_eaten = 0
    while sushi_count > 0: # eat sushi until it's all gone
        with first_chopstick:
            with second_chopstick:
                if sushi_count > 0:
                    sushi_count -= 1
                    sushi_eaten += 1
                    print(name, 'took a piece! Sushi remaining:', sushi_count)
    print(name, 'took', sushi_eaten, 'pieces')

if __name__ == '__main__':
    threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Olivia', chopstick_b, chopstick_c)).start()
    threading.Thread(target=philosopher, args=('Steve', chopstick_a, chopstick_c)).start()


Barron took a piece! Sushi remaining: 4999
Barron took a piece! Sushi remaining: 4998
Barron took a piece! Sushi remaining: 4997
Barron took a piece! Sushi remaining: 4996
Barron took a piece! Sushi remaining: 4995
Barron took a piece! Sushi remaining: 4994
Barron took a piece! Sushi remaining: 4993
Barron took a piece! Sushi remaining: 4992
Barron took a piece! Sushi remaining: 4991
Barron took a piece! Sushi remaining: 4990
Barron took a piece! Sushi remaining: 4989
Barron took a piece! Sushi remaining: 4988
Olivia took a piece! Sushi remaining: 4987
Olivia took a piece! Sushi remaining: 4986
Olivia took a piece! Sushi remaining: 4985
Olivia took a piece! Sushi remaining: 4984
Olivia took a piece! Sushi remaining: 4983
Olivia took a piece! Sushi remaining: 4982
Olivia took a piece! Sushi remaining: 4981
Olivia took a piece! Sushi remaining: 4980
Olivia took a piece! Sushi remaining: 4979
Olivia took a piece! Sushi remaining: 4978
Olivia took a piece! Sushi remaining: 4977
Olivia took

Olivia took a piece! Sushi remaining: 4459
Olivia took a piece! Sushi remaining: 4458
Olivia took a piece! Sushi remaining: 4457
Olivia took a piece! Sushi remaining: 4456
Olivia took a piece! Sushi remaining: 4455
Olivia took a piece! Sushi remaining: 4454
Olivia took a piece! Sushi remaining: 4453
Olivia took a piece! Sushi remaining: 4452
Olivia took a piece! Sushi remaining: 4451
Olivia took a piece! Sushi remaining: 4450
Olivia took a piece! Sushi remaining: 4449
Olivia took a piece! Sushi remaining: 4448
Olivia took a piece! Sushi remaining: 4447
Olivia took a piece! Sushi remaining: 4446
Olivia took a piece! Sushi remaining: 4445
Olivia took a piece! Sushi remaining: 4444
Olivia took a piece! Sushi remaining: 4443
Olivia took a piece! Sushi remaining: 4442
Olivia took a piece! Sushi remaining: 4441
Olivia took a piece! Sushi remaining: 4440
Olivia took a piece! Sushi remaining: 4439
Olivia took a piece! Sushi remaining: 4438
Olivia took a piece! Sushi remaining: 4437
Olivia took

Olivia took a piece! Sushi remaining: 3958
Olivia took a piece! Sushi remaining: 3957
Olivia took a piece! Sushi remaining: 3956
Olivia took a piece! Sushi remaining: 3955
Olivia took a piece! Sushi remaining: 3954
Olivia took a piece! Sushi remaining: 3953
Olivia took a piece! Sushi remaining: 3952
Olivia took a piece! Sushi remaining: 3951
Olivia took a piece! Sushi remaining: 3950
Olivia took a piece! Sushi remaining: 3949
Olivia took a piece! Sushi remaining: 3948
Olivia took a piece! Sushi remaining: 3947
Olivia took a piece! Sushi remaining: 3946
Olivia took a piece! Sushi remaining: 3945
Olivia took a piece! Sushi remaining: 3944
Olivia took a piece! Sushi remaining: 3943
Olivia took a piece! Sushi remaining: 3942
Olivia took a piece! Sushi remaining: 3941
Olivia took a piece! Sushi remaining: 3940
Olivia took a piece! Sushi remaining: 3939
Olivia took a piece! Sushi remaining: 3938
Olivia took a piece! Sushi remaining: 3937
Olivia took a piece! Sushi remaining: 3936
Olivia took

Olivia took a piece! Sushi remaining: 3293
Olivia took a piece! Sushi remaining: 3292
Olivia took a piece! Sushi remaining: 3291
Olivia took a piece! Sushi remaining: 3290
Olivia took a piece! Sushi remaining: 3289
Olivia took a piece! Sushi remaining: 3288
Olivia took a piece! Sushi remaining: 3287
Olivia took a piece! Sushi remaining: 3286
Olivia took a piece! Sushi remaining: 3285
Olivia took a piece! Sushi remaining: 3284
Olivia took a piece! Sushi remaining: 3283
Olivia took a piece! Sushi remaining: 3282
Olivia took a piece! Sushi remaining: 3281
Olivia took a piece! Sushi remaining: 3280
Olivia took a piece! Sushi remaining: 3279
Olivia took a piece! Sushi remaining: 3278
Olivia took a piece! Sushi remaining: 3277
Olivia took a piece! Sushi remaining: 3276
Olivia took a piece! Sushi remaining: 3275
Olivia took a piece! Sushi remaining: 3274
Olivia took a piece! Sushi remaining: 3273
Olivia took a piece! Sushi remaining: 3272
Olivia took a piece! Sushi remaining: 3271
Olivia took

Olivia took a piece! Sushi remaining: 2926
Olivia took a piece! Sushi remaining: 2925
Olivia took a piece! Sushi remaining: 2924
Olivia took a piece! Sushi remaining: 2923
Olivia took a piece! Sushi remaining: 2922
Olivia took a piece! Sushi remaining: 2921
Olivia took a piece! Sushi remaining: 2920
Olivia took a piece! Sushi remaining: 2919
Olivia took a piece! Sushi remaining: 2918
Olivia took a piece! Sushi remaining: 2917
Olivia took a piece! Sushi remaining: 2916
Olivia took a piece! Sushi remaining: 2915
Olivia took a piece! Sushi remaining: 2914
Olivia took a piece! Sushi remaining: 2913
Olivia took a piece! Sushi remaining: 2912
Olivia took a piece! Sushi remaining: 2911
Olivia took a piece! Sushi remaining: 2910
Olivia took a piece! Sushi remaining: 2909
Olivia took a piece! Sushi remaining: 2908
Olivia took a piece! Sushi remaining: 2907
Olivia took a piece! Sushi remaining: 2906
Olivia took a piece! Sushi remaining: 2905
Olivia took a piece! Sushi remaining: 2904
Olivia took

Olivia took a piece! Sushi remaining: 2458
Olivia took a piece! Sushi remaining: 2457
Olivia took a piece! Sushi remaining: 2456
Olivia took a piece! Sushi remaining: 2455
Olivia took a piece! Sushi remaining: 2454
Olivia took a piece! Sushi remaining: 2453
Olivia took a piece! Sushi remaining: 2452
Olivia took a piece! Sushi remaining: 2451
Olivia took a piece! Sushi remaining: 2450
Olivia took a piece! Sushi remaining: 2449
Olivia took a piece! Sushi remaining: 2448
Olivia took a piece! Sushi remaining: 2447
Olivia took a piece! Sushi remaining: 2446
Olivia took a piece! Sushi remaining: 2445
Olivia took a piece! Sushi remaining: 2444
Olivia took a piece! Sushi remaining: 2443
Olivia took a piece! Sushi remaining: 2442
Olivia took a piece! Sushi remaining: 2441
Olivia took a piece! Sushi remaining: 2440
Olivia took a piece! Sushi remaining: 2439
Olivia took a piece! Sushi remaining: 2438
Olivia took a piece! Sushi remaining: 2437
Olivia took a piece! Sushi remaining: 2436
Olivia took

Olivia took a piece! Sushi remaining: 1938
Olivia took a piece! Sushi remaining: 1937
Olivia took a piece! Sushi remaining: 1936
Olivia took a piece! Sushi remaining: 1935
Olivia took a piece! Sushi remaining: 1934
Olivia took a piece! Sushi remaining: 1933
Olivia took a piece! Sushi remaining: 1932
Olivia took a piece! Sushi remaining: 1931
Olivia took a piece! Sushi remaining: 1930
Olivia took a piece! Sushi remaining: 1929
Olivia took a piece! Sushi remaining: 1928
Olivia took a piece! Sushi remaining: 1927
Olivia took a piece! Sushi remaining: 1926
Olivia took a piece! Sushi remaining: 1925
Olivia took a piece! Sushi remaining: 1924
Olivia took a piece! Sushi remaining: 1923
Olivia took a piece! Sushi remaining: 1922
Olivia took a piece! Sushi remaining: 1921
Olivia took a piece! Sushi remaining: 1920
Olivia took a piece! Sushi remaining: 1919
Olivia took a piece! Sushi remaining: 1918
Olivia took a piece! Sushi remaining: 1917
Olivia took a piece! Sushi remaining: 1916
Olivia took

Olivia took a piece! Sushi remaining: 1343
Olivia took a piece! Sushi remaining: 1342
Olivia took a piece! Sushi remaining: 1341
Olivia took a piece! Sushi remaining: 1340
Olivia took a piece! Sushi remaining: 1339
Olivia took a piece! Sushi remaining: 1338
Olivia took a piece! Sushi remaining: 1337
Olivia took a piece! Sushi remaining: 1336
Olivia took a piece! Sushi remaining: 1335
Olivia took a piece! Sushi remaining: 1334
Olivia took a piece! Sushi remaining: 1333
Olivia took a piece! Sushi remaining: 1332
Olivia took a piece! Sushi remaining: 1331
Olivia took a piece! Sushi remaining: 1330
Olivia took a piece! Sushi remaining: 1329
Olivia took a piece! Sushi remaining: 1328
Olivia took a piece! Sushi remaining: 1327
Olivia took a piece! Sushi remaining: 1326
Olivia took a piece! Sushi remaining: 1325
Olivia took a piece! Sushi remaining: 1324
Olivia took a piece! Sushi remaining: 1323
Olivia took a piece! Sushi remaining: 1322
Olivia took a piece! Sushi remaining: 1321
Olivia took

Olivia took a piece! Sushi remaining: 938
Olivia took a piece! Sushi remaining: 937
Olivia took a piece! Sushi remaining: 936
Olivia took a piece! Sushi remaining: 935
Olivia took a piece! Sushi remaining: 934
Olivia took a piece! Sushi remaining: 933
Olivia took a piece! Sushi remaining: 932
Olivia took a piece! Sushi remaining: 931
Olivia took a piece! Sushi remaining: 930
Olivia took a piece! Sushi remaining: 929
Olivia took a piece! Sushi remaining: 928
Olivia took a piece! Sushi remaining: 927
Olivia took a piece! Sushi remaining: 926
Olivia took a piece! Sushi remaining: 925
Olivia took a piece! Sushi remaining: 924
Olivia took a piece! Sushi remaining: 923
Olivia took a piece! Sushi remaining: 922
Olivia took a piece! Sushi remaining: 921
Olivia took a piece! Sushi remaining: 920
Olivia took a piece! Sushi remaining: 919
Olivia took a piece! Sushi remaining: 918
Olivia took a piece! Sushi remaining: 917
Olivia took a piece! Sushi remaining: 916
Olivia took a piece! Sushi remaini

Olivia took a piece! Sushi remaining: 438
Olivia took a piece! Sushi remaining: 437
Olivia took a piece! Sushi remaining: 436
Olivia took a piece! Sushi remaining: 435
Olivia took a piece! Sushi remaining: 434
Olivia took a piece! Sushi remaining: 433
Olivia took a piece! Sushi remaining: 432
Olivia took a piece! Sushi remaining: 431
Olivia took a piece! Sushi remaining: 430
Olivia took a piece! Sushi remaining: 429
Olivia took a piece! Sushi remaining: 428
Olivia took a piece! Sushi remaining: 427
Olivia took a piece! Sushi remaining: 426
Olivia took a piece! Sushi remaining: 425
Olivia took a piece! Sushi remaining: 424
Olivia took a piece! Sushi remaining: 423
Olivia took a piece! Sushi remaining: 422
Olivia took a piece! Sushi remaining: 421
Olivia took a piece! Sushi remaining: 420
Olivia took a piece! Sushi remaining: 419
Olivia took a piece! Sushi remaining: 418
Olivia took a piece! Sushi remaining: 417
Olivia took a piece! Sushi remaining: 416
Olivia took a piece! Sushi remaini

When it finishes, we see that each of the philosophers got a different amount of sushi and it's not particularly fair. Olivia took way more sushi than Barron or Steve. She has over 4000 of the pieces. But it's not because she's greedy. In this case, it's because of the order in which the 3 philosophers are taking chopsticks. Barron and Steve are both competing for `chopostick_a` as their first chopstick, but Olivia is the only philosopher going for `chopstick_b` first. Since she has less competition to get her first chopstick, she ends up getting to take sushi more frequently.

To make things fair for this example, we'll make all of the philosophers acquire the same two chopsticks, `chopostick_a` first and then `chopstick_b`.

In [6]:
#!/usr/bin/env python3
""" Three philosophers, thinking and eating sushi """

import threading

chopstick_a = threading.Lock()
chopstick_b = threading.Lock()
chopstick_c = threading.Lock()
sushi_count = 5000

def philosopher(name, first_chopstick, second_chopstick):
    global sushi_count
    sushi_eaten = 0
    while sushi_count > 0: # eat sushi until it's all gone
        with first_chopstick:
            with second_chopstick:
                if sushi_count > 0:
                    sushi_count -= 1
                    sushi_eaten += 1
                    print(name, 'took a piece! Sushi remaining:', sushi_count)
    print(name, 'took', sushi_eaten, 'pieces')

if __name__ == '__main__':
    threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Olivia', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Steve', chopstick_a, chopstick_b)).start()


Barron took a piece! Sushi remaining: 4999
Barron took a piece! Sushi remaining: 4998
Barron took a piece! Sushi remaining: 4997
Barron took a piece! Sushi remaining: 4996
Barron took a piece! Sushi remaining: 4995
Barron took a piece! Sushi remaining: 4994
Barron took a piece! Sushi remaining: 4993
Barron took a piece! Sushi remaining: 4992
Barron took a piece! Sushi remaining: 4991
Barron took a piece! Sushi remaining: 4990
Barron took a piece! Sushi remaining: 4989
Barron took a piece! Sushi remaining: 4988
Barron took a piece! Sushi remaining: 4987
Barron took a piece! Sushi remaining: 4986
Barron took a piece! Sushi remaining: 4985
Barron took a piece! Sushi remaining: 4984
Barron took a piece! Sushi remaining: 4983
Barron took a piece! Sushi remaining: 4982
Barron took a piece! Sushi remaining: 4981
Barron took a piece! Sushi remaining: 4980
Barron took a piece! Sushi remaining: 4979
Barron took a piece! Sushi remaining: 4978
Barron took a piece! Sushi remaining: 4977
Barron took

Barron took a piece! Sushi remaining: 4344
Barron took a piece! Sushi remaining: 4343
Barron took a piece! Sushi remaining: 4342
Barron took a piece! Sushi remaining: 4341
Barron took a piece! Sushi remaining: 4340
Barron took a piece! Sushi remaining: 4339
Barron took a piece! Sushi remaining: 4338
Barron took a piece! Sushi remaining: 4337
Barron took a piece! Sushi remaining: 4336
Barron took a piece! Sushi remaining: 4335
Barron took a piece! Sushi remaining: 4334
Barron took a piece! Sushi remaining: 4333
Barron took a piece! Sushi remaining: 4332
Barron took a piece! Sushi remaining: 4331
Barron took a piece! Sushi remaining: 4330
Barron took a piece! Sushi remaining: 4329
Barron took a piece! Sushi remaining: 4328
Olivia took a piece! Sushi remaining: 4327
Olivia took a piece! Sushi remaining: 4326
Olivia took a piece! Sushi remaining: 4325
Olivia took a piece! Sushi remaining: 4324
Olivia took a piece! Sushi remaining: 4323
Olivia took a piece! Sushi remaining: 4322
Olivia took

Olivia took a piece! Sushi remaining: 3994
Olivia took a piece! Sushi remaining: 3993
Olivia took a piece! Sushi remaining: 3992
Olivia took a piece! Sushi remaining: 3991
Olivia took a piece! Sushi remaining: 3990
Olivia took a piece! Sushi remaining: 3989
Olivia took a piece! Sushi remaining: 3988
Olivia took a piece! Sushi remaining: 3987
Olivia took a piece! Sushi remaining: 3986
Olivia took a piece! Sushi remaining: 3985
Olivia took a piece! Sushi remaining: 3984
Olivia took a piece! Sushi remaining: 3983
Olivia took a piece! Sushi remaining: 3982
Olivia took a piece! Sushi remaining: 3981
Olivia took a piece! Sushi remaining: 3980
Olivia took a piece! Sushi remaining: 3979
Olivia took a piece! Sushi remaining: 3978
Olivia took a piece! Sushi remaining: 3977
Olivia took a piece! Sushi remaining: 3976
Olivia took a piece! Sushi remaining: 3975
Olivia took a piece! Sushi remaining: 3974
Olivia took a piece! Sushi remaining: 3973
Olivia took a piece! Sushi remaining: 3972
Olivia took

Olivia took a piece! Sushi remaining: 3494
Olivia took a piece! Sushi remaining: 3493
Olivia took a piece! Sushi remaining: 3492
Olivia took a piece! Sushi remaining: 3491
Olivia took a piece! Sushi remaining: 3490
Olivia took a piece! Sushi remaining: 3489
Olivia took a piece! Sushi remaining: 3488
Olivia took a piece! Sushi remaining: 3487
Olivia took a piece! Sushi remaining: 3486
Olivia took a piece! Sushi remaining: 3485
Olivia took a piece! Sushi remaining: 3484
Olivia took a piece! Sushi remaining: 3483
Olivia took a piece! Sushi remaining: 3482
Olivia took a piece! Sushi remaining: 3481
Olivia took a piece! Sushi remaining: 3480
Olivia took a piece! Sushi remaining: 3479
Olivia took a piece! Sushi remaining: 3478
Olivia took a piece! Sushi remaining: 3477
Olivia took a piece! Sushi remaining: 3476
Olivia took a piece! Sushi remaining: 3475
Olivia took a piece! Sushi remaining: 3474
Olivia took a piece! Sushi remaining: 3473
Olivia took a piece! Sushi remaining: 3472
Olivia took

Olivia took a piece! Sushi remaining: 2724
Olivia took a piece! Sushi remaining: 2723
Olivia took a piece! Sushi remaining: 2722
Olivia took a piece! Sushi remaining: 2721
Olivia took a piece! Sushi remaining: 2720
Olivia took a piece! Sushi remaining: 2719
Olivia took a piece! Sushi remaining: 2718
Olivia took a piece! Sushi remaining: 2717
Olivia took a piece! Sushi remaining: 2716
Olivia took a piece! Sushi remaining: 2715
Olivia took a piece! Sushi remaining: 2714
Olivia took a piece! Sushi remaining: 2713
Olivia took a piece! Sushi remaining: 2712
Olivia took a piece! Sushi remaining: 2711
Olivia took a piece! Sushi remaining: 2710
Olivia took a piece! Sushi remaining: 2709
Olivia took a piece! Sushi remaining: 2708
Olivia took a piece! Sushi remaining: 2707
Olivia took a piece! Sushi remaining: 2706
Olivia took a piece! Sushi remaining: 2705
Olivia took a piece! Sushi remaining: 2704
Olivia took a piece! Sushi remaining: 2703
Olivia took a piece! Sushi remaining: 2702
Olivia took

Olivia took a piece! Sushi remaining: 2492
Olivia took a piece! Sushi remaining: 2491
Olivia took a piece! Sushi remaining: 2490
Olivia took a piece! Sushi remaining: 2489
Olivia took a piece! Sushi remaining: 2488
Barron took a piece! Sushi remaining: 2487
Barron took a piece! Sushi remaining: 2486
Barron took a piece! Sushi remaining: 2485
Barron took a piece! Sushi remaining: 2484
Barron took a piece! Sushi remaining: 2483
Barron took a piece! Sushi remaining: 2482
Barron took a piece! Sushi remaining: 2481
Barron took a piece! Sushi remaining: 2480
Barron took a piece! Sushi remaining: 2479
Barron took a piece! Sushi remaining: 2478
Barron took a piece! Sushi remaining: 2477
Barron took a piece! Sushi remaining: 2476
Barron took a piece! Sushi remaining: 2475
Steve took a piece! Sushi remaining: 2474
Steve took a piece! Sushi remaining: 2473
Steve took a piece! Sushi remaining: 2472
Steve took a piece! Sushi remaining: 2471
Steve took a piece! Sushi remaining: 2470
Steve took a pie

Steve took a piece! Sushi remaining: 1891
Steve took a piece! Sushi remaining: 1890
Steve took a piece! Sushi remaining: 1889
Steve took a piece! Sushi remaining: 1888
Steve took a piece! Sushi remaining: 1887
Steve took a piece! Sushi remaining: 1886
Steve took a piece! Sushi remaining: 1885
Steve took a piece! Sushi remaining: 1884
Steve took a piece! Sushi remaining: 1883
Steve took a piece! Sushi remaining: 1882
Steve took a piece! Sushi remaining: 1881
Steve took a piece! Sushi remaining: 1880
Steve took a piece! Sushi remaining: 1879
Steve took a piece! Sushi remaining: 1878
Steve took a piece! Sushi remaining: 1877
Steve took a piece! Sushi remaining: 1876
Steve took a piece! Sushi remaining: 1875
Steve took a piece! Sushi remaining: 1874
Steve took a piece! Sushi remaining: 1873
Steve took a piece! Sushi remaining: 1872
Steve took a piece! Sushi remaining: 1871
Steve took a piece! Sushi remaining: 1870
Steve took a piece! Sushi remaining: 1869
Steve took a piece! Sushi remainin

Steve took a piece! Sushi remaining: 1328
Steve took a piece! Sushi remaining: 1327
Steve took a piece! Sushi remaining: 1326
Steve took a piece! Sushi remaining: 1325
Steve took a piece! Sushi remaining: 1324
Steve took a piece! Sushi remaining: 1323
Steve took a piece! Sushi remaining: 1322
Steve took a piece! Sushi remaining: 1321
Steve took a piece! Sushi remaining: 1320
Steve took a piece! Sushi remaining: 1319
Steve took a piece! Sushi remaining: 1318
Steve took a piece! Sushi remaining: 1317
Steve took a piece! Sushi remaining: 1316
Steve took a piece! Sushi remaining: 1315
Steve took a piece! Sushi remaining: 1314
Steve took a piece! Sushi remaining: 1313
Steve took a piece! Sushi remaining: 1312
Steve took a piece! Sushi remaining: 1311
Steve took a piece! Sushi remaining: 1310
Steve took a piece! Sushi remaining: 1309
Steve took a piece! Sushi remaining: 1308
Steve took a piece! Sushi remaining: 1307
Steve took a piece! Sushi remaining: 1306
Steve took a piece! Sushi remainin

Steve took a piece! Sushi remaining: 828
Steve took a piece! Sushi remaining: 827
Steve took a piece! Sushi remaining: 826
Steve took a piece! Sushi remaining: 825
Steve took a piece! Sushi remaining: 824
Steve took a piece! Sushi remaining: 823
Steve took a piece! Sushi remaining: 822
Steve took a piece! Sushi remaining: 821
Steve took a piece! Sushi remaining: 820
Steve took a piece! Sushi remaining: 819
Steve took a piece! Sushi remaining: 818
Steve took a piece! Sushi remaining: 817
Steve took a piece! Sushi remaining: 816
Steve took a piece! Sushi remaining: 815
Steve took a piece! Sushi remaining: 814
Steve took a piece! Sushi remaining: 813
Steve took a piece! Sushi remaining: 812
Steve took a piece! Sushi remaining: 811
Steve took a piece! Sushi remaining: 810
Steve took a piece! Sushi remaining: 809
Steve took a piece! Sushi remaining: 808
Steve took a piece! Sushi remaining: 807
Steve took a piece! Sushi remaining: 806
Steve took a piece! Sushi remaining: 805
Steve took a pie

Steve took a piece! Sushi remaining: 175
Steve took a piece! Sushi remaining: 174
Steve took a piece! Sushi remaining: 173
Steve took a piece! Sushi remaining: 172
Steve took a piece! Sushi remaining: 171
Steve took a piece! Sushi remaining: 170
Steve took a piece! Sushi remaining: 169
Steve took a piece! Sushi remaining: 168
Steve took a piece! Sushi remaining: 167
Steve took a piece! Sushi remaining: 166
Steve took a piece! Sushi remaining: 165
Steve took a piece! Sushi remaining: 164
Steve took a piece! Sushi remaining: 163
Steve took a piece! Sushi remaining: 162
Steve took a piece! Sushi remaining: 161
Steve took a piece! Sushi remaining: 160
Steve took a piece! Sushi remaining: 159
Steve took a piece! Sushi remaining: 158
Steve took a piece! Sushi remaining: 157
Steve took a piece! Sushi remaining: 156
Steve took a piece! Sushi remaining: 155
Steve took a piece! Sushi remaining: 154
Steve took a piece! Sushi remaining: 153
Steve took a piece! Sushi remaining: 152
Steve took a pie

Let's see what happens if we drastically increase the number of philosophers at the dinner party using a `for` loop to create 50 instances of Barron, Olivia, and Steve. In this case, we'll have 150 philosophers competing for 5000 pieces of sushi.

In [7]:
#!/usr/bin/env python3
""" Three philosophers, thinking and eating sushi """

import threading

chopstick_a = threading.Lock()
chopstick_b = threading.Lock()
chopstick_c = threading.Lock()
sushi_count = 5000

def philosopher(name, first_chopstick, second_chopstick):
    global sushi_count
    sushi_eaten = 0
    while sushi_count > 0: # eat sushi until it's all gone
        with first_chopstick:
            with second_chopstick:
                if sushi_count > 0:
                    sushi_count -= 1
                    sushi_eaten += 1
                    print(name, 'took a piece! Sushi remaining:', sushi_count)
    print(name, 'took', sushi_eaten, 'pieces')

if __name__ == '__main__':
    for thread in range(50):
        threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
        threading.Thread(target=philosopher, args=('Olivia', chopstick_a, chopstick_b)).start()
        threading.Thread(target=philosopher, args=('Steve', chopstick_a, chopstick_b)).start()


Barron took a piece! Sushi remaining: 4999
Barron took a piece! Sushi remaining: 4998
Barron took a piece! Sushi remaining: 4997
Barron took a piece! Sushi remaining: 4996
Barron took a piece! Sushi remaining: 4995
Barron took a piece! Sushi remaining: 4994
Barron took a piece! Sushi remaining: 4993
Barron took a piece! Sushi remaining: 4992
Barron took a piece! Sushi remaining: 4991
Barron took a piece! Sushi remaining: 4990
Barron took a piece! Sushi remaining: 4989
Barron took a piece! Sushi remaining: 4988
Barron took a piece! Sushi remaining: 4987
Barron took a piece! Sushi remaining: 4986
Barron took a piece! Sushi remaining: 4985
Barron took a piece! Sushi remaining: 4984
Barron took a piece! Sushi remaining: 4983
Barron took a piece! Sushi remaining: 4982
Barron took a piece! Sushi remaining: 4981
Barron took a piece! Sushi remaining: 4980
Barron took a piece! Sushi remaining: 4979
Barron took a piece! Sushi remaining: 4978
Barron took a piece! Sushi remaining: 4977
Barron took

Steve took a piece! Sushi remaining: 4485
Steve took a piece! Sushi remaining: 4484
Steve took a piece! Sushi remaining: 4483
Steve took a piece! Sushi remaining: 4482
Steve took a piece! Sushi remaining: 4481
Steve took a piece! Sushi remaining: 4480
Steve took a piece! Sushi remaining: 4479
Steve took a piece! Sushi remaining: 4478
Steve took a piece! Sushi remaining: 4477
Steve took a piece! Sushi remaining: 4476
Steve took a piece! Sushi remaining: 4475
Steve took a piece! Sushi remaining: 4474
Steve took a piece! Sushi remaining: 4473
Steve took a piece! Sushi remaining: 4472
Steve took a piece! Sushi remaining: 4471
Steve took a piece! Sushi remaining: 4470
Steve took a piece! Sushi remaining: 4469
Steve took a piece! Sushi remaining: 4468
Steve took a piece! Sushi remaining: 4467
Steve took a piece! Sushi remaining: 4466
Steve took a piece! Sushi remaining: 4465
Steve took a piece! Sushi remaining: 4464
Steve took a piece! Sushi remaining: 4463
Steve took a piece! Sushi remainin

Barron took a piece! Sushi remaining: 4058
Barron took a piece! Sushi remaining: 4057
Barron took a piece! Sushi remaining: 4056
Barron took a piece! Sushi remaining: 4055
Barron took a piece! Sushi remaining: 4054
Barron took a piece! Sushi remaining: 4053
Barron took a piece! Sushi remaining: 4052
Barron took a piece! Sushi remaining: 4051
Barron took a piece! Sushi remaining: 4050
Barron took a piece! Sushi remaining: 4049
Barron took a piece! Sushi remaining: 4048
Barron took a piece! Sushi remaining: 4047
Barron took a piece! Sushi remaining: 4046
Barron took a piece! Sushi remaining: 4045
Barron took a piece! Sushi remaining: 4044
Barron took a piece! Sushi remaining: 4043
Barron took a piece! Sushi remaining: 4042
Barron took a piece! Sushi remaining: 4041
Barron took a piece! Sushi remaining: 4040
Barron took a piece! Sushi remaining: 4039
Barron took a piece! Sushi remaining: 4038
Barron took a piece! Sushi remaining: 4037
Barron took a piece! Sushi remaining: 4036
Barron took

Barron took a piece! Sushi remaining: 3861
Barron took a piece! Sushi remaining: 3860
Barron took a piece! Sushi remaining: 3859
Barron took a piece! Sushi remaining: 3858
Barron took a piece! Sushi remaining: 3857
Barron took a piece! Sushi remaining: 3856
Barron took a piece! Sushi remaining: 3855
Barron took a piece! Sushi remaining: 3854
Barron took a piece! Sushi remaining: 3853
Barron took a piece! Sushi remaining: 3852
Barron took a piece! Sushi remaining: 3851
Barron took a piece! Sushi remaining: 3850
Barron took a piece! Sushi remaining: 3849
Barron took a piece! Sushi remaining: 3848
Barron took a piece! Sushi remaining: 3847
Barron took a piece! Sushi remaining: 3846
Barron took a piece! Sushi remaining: 3845
Barron took a piece! Sushi remaining: 3844
Barron took a piece! Sushi remaining: 3843
Barron took a piece! Sushi remaining: 3842
Barron took a piece! Sushi remaining: 3841
Barron took a piece! Sushi remaining: 3840
Barron took a piece! Sushi remaining: 3839
Barron took

Steve took a piece! Sushi remaining: 3493
Steve took a piece! Sushi remaining: 3492
Steve took a piece! Sushi remaining: 3491
Steve took a piece! Sushi remaining: 3490
Steve took a piece! Sushi remaining: 3489
Steve took a piece! Sushi remaining: 3488
Steve took a piece! Sushi remaining: 3487
Steve took a piece! Sushi remaining: 3486
Steve took a piece! Sushi remaining: 3485
Steve took a piece! Sushi remaining: 3484
Steve took a piece! Sushi remaining: 3483
Steve took a piece! Sushi remaining: 3482
Steve took a piece! Sushi remaining: 3481
Steve took a piece! Sushi remaining: 3480
Steve took a piece! Sushi remaining: 3479
Steve took a piece! Sushi remaining: 3478
Steve took a piece! Sushi remaining: 3477
Steve took a piece! Sushi remaining: 3476
Steve took a piece! Sushi remaining: 3475
Steve took a piece! Sushi remaining: 3474
Steve took a piece! Sushi remaining: 3473
Steve took a piece! Sushi remaining: 3472
Steve took a piece! Sushi remaining: 3471
Steve took a piece! Sushi remainin

...

Olivia took a piece! Sushi remaining: 1927
Olivia took a piece! Sushi remaining: 1926
Olivia took a piece! Sushi remaining: 1925
Olivia took a piece! Sushi remaining: 1924
Olivia took a piece! Sushi remaining: 1923
Olivia took a piece! Sushi remaining: 1922
Olivia took a piece! Sushi remaining: 1921
Olivia took a piece! Sushi remaining: 1920
Olivia took a piece! Sushi remaining: 1919
Olivia took a piece! Sushi remaining: 1918
Olivia took a piece! Sushi remaining: 1917
Olivia took a piece! Sushi remaining: 1916
Olivia took a piece! Sushi remaining: 1915
Olivia took a piece! Sushi remaining: 1914
Olivia took a piece! Sushi remaining: 1913
Olivia took a piece! Sushi remaining: 1912
Olivia took a piece! Sushi remaining: 1911
Olivia took a piece! Sushi remaining: 1910
Olivia took a piece! Sushi remaining: 1909
Olivia took a piece! Sushi remaining: 1908
Olivia took a piece! Sushi remaining: 1907
Olivia took a piece! Sushi remaining: 1906
Olivia took a piece! Sushi remaining: 1905
Olivia took

Steve took a piece! Sushi remaining: 1381
Steve took a piece! Sushi remaining: 1380
Steve took a piece! Sushi remaining: 1379
Steve took a piece! Sushi remaining: 1378
Steve took a piece! Sushi remaining: 1377
Steve took a piece! Sushi remaining: 1376
Steve took a piece! Sushi remaining: 1375
Steve took a piece! Sushi remaining: 1374
Steve took a piece! Sushi remaining: 1373
Steve took a piece! Sushi remaining: 1372
Steve took a piece! Sushi remaining: 1371
Steve took a piece! Sushi remaining: 1370
Steve took a piece! Sushi remaining: 1369
Steve took a piece! Sushi remaining: 1368
Steve took a piece! Sushi remaining: 1367
Steve took a piece! Sushi remaining: 1366
Steve took a piece! Sushi remaining: 1365
Steve took a piece! Sushi remaining: 1364
Steve took a piece! Sushi remaining: 1363
Steve took a piece! Sushi remaining: 1362
Steve took a piece! Sushi remaining: 1361
Steve took a piece! Sushi remaining: 1360
Steve took a piece! Sushi remaining: 1359
Steve took a piece! Sushi remainin

Steve took a piece! Sushi remaining: 831
Steve took a piece! Sushi remaining: 830
Steve took a piece! Sushi remaining: 829
Steve took a piece! Sushi remaining: 828
Steve took a piece! Sushi remaining: 827
Steve took a piece! Sushi remaining: 826
Steve took a piece! Sushi remaining: 825
Steve took a piece! Sushi remaining: 824
Steve took a piece! Sushi remaining: 823
Steve took a piece! Sushi remaining: 822
Steve took a piece! Sushi remaining: 821
Steve took a piece! Sushi remaining: 820
Steve took a piece! Sushi remaining: 819
Steve took a piece! Sushi remaining: 818
Steve took a piece! Sushi remaining: 817
Steve took a piece! Sushi remaining: 816
Steve took a piece! Sushi remaining: 815
Steve took a piece! Sushi remaining: 814
Steve took a piece! Sushi remaining: 813
Steve took a piece! Sushi remaining: 812
Steve took a piece! Sushi remaining: 811
Steve took a piece! Sushi remaining: 810
Steve took a piece! Sushi remaining: 809
Steve took a piece! Sushi remaining: 808
Steve took a pie

Steve took a piece! Sushi remaining: 385
Steve took a piece! Sushi remaining: 384
Steve took a piece! Sushi remaining: 383
Steve took a piece! Sushi remaining: 382
Steve took a piece! Sushi remaining: 381
Steve took a piece! Sushi remaining: 380
Steve took a piece! Sushi remaining: 379
Steve took a piece! Sushi remaining: 378
Steve took a piece! Sushi remaining: 377
Steve took a piece! Sushi remaining: 376
Steve took a piece! Sushi remaining: 375
Steve took a piece! Sushi remaining: 374
Steve took a piece! Sushi remaining: 373
Steve took a piece! Sushi remaining: 372
Steve took a piece! Sushi remaining: 371
Steve took a piece! Sushi remaining: 370
Steve took a piece! Sushi remaining: 369
Steve took a piece! Sushi remaining: 368
Steve took a piece! Sushi remaining: 367
Steve took a piece! Sushi remaining: 366
Steve took a piece! Sushi remaining: 365
Steve took a piece! Sushi remaining: 364
Steve took a piece! Sushi remaining: 363
Steve took a piece! Sushi remaining: 362
Steve took a pie

 took 779 Barron took Steve took0 pieces
 0 pieces
Olivia took 0 pieces
Steve took 0 pieces
pieces
Steve took 0 pieces
Barron tookSteve took 0  pieces
0 pieces
Steve tookBarron took Olivia took0 pieces
  0Steve took 0 pieces
0 pieces
 pieces
Olivia took 0 pieces
Barron took 0 pieces
Barron took 0 pieces
Olivia took Olivia took 0 Steve took 0 pieces
0pieces
 pieces
Steve took Steve took 0 pieces
Barron0 took 0 pieces
Barron took 0 pieces
 pieces
Olivia took 0 pieces
Steve took 0 pieces
Barron took 0 pieces
Barron took 0 pieces
Steve took 0 BarronpiecesOlivia took 0 pieces
 
OliviatookOlivia tookOlivia took Steve took   took 219 pieces
 0 pieces
Steve0 piecesBarron took0 pieces
Steve took 0
BarronBarron took 0 pieces
   took 104 pieces
0 piecespieces
0
 Barron took tookSteve took 0 piecesSteve took 
Barron  pieces36Olivia took0
 Barron took 0  piecesSteve 0piecespieces
 took 0 pieces
 0 pieces
Barron took
took 0 pieces
 
Steve took 0 Olivia tookpiecespieces 
Steve took0Steve  0 pieces
 t

We can see that some philosophers got a lot of sushi compared to others. Many of these philosophers starve completely taking 0. All of these threads were created with the same default priority, but because there're so many, some threads never got a chance to execute. This represents a real problem in programs containing a large number of threads. For example, if instead of feeding sushi to a bunch of philosophers, this program was a web server that created new threads to handle a huge number of incoming requests, some of those requests may never get processed that would lead to some very impatient and angry users on the other end. Some techniques can be used to improve or even guarantee fairness among threads, but that type of workload management is very situation dependent.